In [1]:
import numpy as np
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm, norm

In [2]:
def is_unitary(m):
    return norm(np.eye(len(m)) - m.dot(m.T.conj()))
def is_hermitian(m):
    return norm(m-m.T.conj())

In [60]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])

# coefficients = [1]*3
# bases = ['YZ', 'ZX', 'YX'] # these are all commuting
hamiltonian = np.zeros((2**len(bases[0]), 2**len(bases[0])), dtype=np.cdouble)
# comment this out when stuff actually works
for c, lbl in zip(coefficients, bases):
    op = Operator(Pauli(label=lbl))
    hamiltonian+=c*op.data

In [54]:
is_hermitian(hamiltonian)

0.0

In [55]:

def get_circuit(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    for i in range(n):
        for c, b in zip(coefficients, bases):
            # implement e^{-i*c*b*t/n}
            b = b[::-1]
            q = []
            qc.barrier()
            for i, op in enumerate(b):
                if (op == 'X'):
                    qc.h(i)
                    q.append(i)
                elif (op == 'Z'):
                    q.append(i)
                elif (op == 'Y'):
                    qc.sdg(i)
                    qc.h(i)
                    q.append(i)
                elif (op == 'I'):
                    continue
            qc.barrier()
            if (len(q) > 0):
                if (len(q) > 1):
                    for i in range(len(q)-1):
                        qc.cx(q[i], q[-1])
                qc.rz(2*c*t/n, q[-1])
                if (len(q) > 1):
                    for i in reversed(range(len(q)-1)):
                        qc.cx(q[i], q[-1])
                qc.barrier()
                for i, op in enumerate(b):
                    if (op == 'X'):
                        qc.h(i)
                    elif (op == 'Z'):
                        continue
                    elif (op == 'Y'):
                        qc.h(i)
                        qc.s(i)
                    elif (op == 'I'):
                        continue
            qc.barrier()
    return qc

def get_unitaries(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [56]:
get_circuit(n=1).draw()

░              ░                     ░            ░  ░ ┌───┐ ░      »
q_0: ─░──────────────░───■─────────────■───░────────────░──░─┤ H ├─░───■──»
      ░ ┌─────┐┌───┐ ░ ┌─┴─┐┌───────┐┌─┴─┐ ░ ┌───┐┌───┐ ░  ░ └───┘ ░ ┌─┴─┐»
q_1: ─░─┤ Sdg ├┤ H ├─░─┤ X ├┤ Rz(2) ├┤ X ├─░─┤ H ├┤ S ├─░──░───────░─┤ X ├»
      ░ └─────┘└───┘ ░ └───┘└───────┘└───┘ ░ └───┘└───┘ ░  ░       ░ └───┘»
«                    ░ ┌───┐ ░  ░  ┌───┐       ░                     ░ ┌───┐»
«q_0: ───────────■───░─┤ H ├─░──░──┤ H ├───────░───■─────────────■───░─┤ H ├»
«     ┌───────┐┌─┴─┐ ░ └───┘ ░  ░ ┌┴───┴┐┌───┐ ░ ┌─┴─┐┌───────┐┌─┴─┐ ░ ├───┤»
«q_1: ┤ Rz(2) ├┤ X ├─░───────░──░─┤ Sdg ├┤ H ├─░─┤ X ├┤ Rz(2) ├┤ X ├─░─┤ H ├»
«     └───────┘└───┘ ░       ░  ░ └─────┘└───┘ ░ └───┘└───────┘└───┘ ░ └───┘»
«           ░ 
«q_0: ──────░─
«     ┌───┐ ░ 
«q_1: ┤ S ├─░─
«     └───┘ ░

In [62]:
e_ith, U_circuit = get_unitaries(n=4)

In [59]:
def get_error(e_ith, U_circuit):
    diff = U_circuit.data - e_ith
    # measure the operator's accuracy
    w, v = LA.eig(diff)

    return np.absolute(max(w))

get_error(e_ith, U_circuit)

0.0140768368731666

In [18]:
np.around(e_ith, decimals=3)

array([[ 0.54 +0.j,  0.   +0.j,  0.   +0.j, -0.841+0.j],
       [ 0.   +0.j,  0.54 +0.j,  0.841+0.j,  0.   +0.j],
       [ 0.   +0.j, -0.841+0.j,  0.54 +0.j,  0.   +0.j],
       [ 0.841+0.j,  0.   +0.j,  0.   +0.j,  0.54 +0.j]])

In [32]:
np.around(expm(hamiltonian), decimals=3)

array([[1.543+0.j   , 0.   +0.j   , 0.   +0.j   , 0.   -1.175j],
       [0.   +0.j   , 1.543+0.j   , 0.   +1.175j, 0.   +0.j   ],
       [0.   +0.j   , 0.   -1.175j, 1.543+0.j   , 0.   +0.j   ],
       [0.   +1.175j, 0.   +0.j   , 0.   +0.j   , 1.543+0.j   ]])

In [20]:
hamiltonian

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
       [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [17]:
np.around(U_circuit, decimals=3)

array([[ 0.54-0.j   ,  0.  +0.j   ,  0.  +0.j   , -0.  -0.841j],
       [ 0.  +0.j   ,  0.54+0.j   , -0.  -0.841j,  0.  +0.j   ],
       [ 0.  +0.j   , -0.  -0.841j,  0.54-0.j   ,  0.  +0.j   ],
       [-0.  -0.841j,  0.  +0.j   ,  0.  +0.j   ,  0.54-0.j   ]])

In [126]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [124]:
e_ith-U_circuit.data

array([[-1.50322374e+00+2.45449949e-16j, -5.87530474e-01-6.12876819e-17j,
         9.27359830e-19-4.88601814e-01j,  1.21911489e-18-3.67379553e-01j],
       [-5.87730457e-01-6.12360182e-17j, -3.27962806e-01+3.68501279e-16j,
         2.45226529e-18-3.67377553e-01j,  2.54543933e-17+2.46155292e-01j],
       [ 9.27359830e-19-4.88601814e-01j,  3.67902959e-18-3.67379553e-01j,
        -1.50322374e+00+6.13325847e-16j, -5.87530474e-01-6.12876819e-17j],
       [ 4.90553476e-18-3.67377553e-01j,  2.54543933e-17+2.46155292e-01j,
        -5.87730457e-01-6.12360182e-17j, -3.27962806e-01+7.34651747e-16j]])

In [15]:
np.unique(e_ith4-U_circuit4.data), np.unique(e_ith4-U_circuit4.data)

(array([-1.11022302e-16+0.j        , -5.15252374e-17+0.84147098j,
         0.00000000e+00-0.84147098j,  0.00000000e+00+0.j        ,
         5.15252374e-17+0.84147098j]),
 array([-1.11022302e-16+0.j        , -5.15252374e-17+0.84147098j,
         0.00000000e+00-0.84147098j,  0.00000000e+00+0.j        ,
         5.15252374e-17+0.84147098j]))

10 qubits -> 2^10 states -> 2^20 entries ($\approx$ 1,000,000, which is quite reasonable)